In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('NLP_Spam_Filter').getOrCreate()

In [4]:
data = spark.read.csv('../../SparkData/SMSSpamCollection', inferSchema=True,
                     sep='\t')

In [6]:
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [7]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [9]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import length

In [11]:
data = data.withColumn('length', length(data['text']))

In [13]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [14]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



The average length of a spam message is much longer than a ham message.

In [15]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover,
                               CountVectorizer, IDF, StringIndexer)

In [16]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], 
                           outputCol='features')

In [19]:
from pyspark.ml.classification import NaiveBayes

In [20]:
nb = NaiveBayes()

In [21]:
from pyspark.ml import Pipeline

In [22]:
data_pred_pipe = Pipeline(stages= [
    ham_spam_to_numeric,
    tokenizer,
    stop_remove,
    count_vec,
    idf,
    clean_up
])

In [23]:
cleaner = data_pred_pipe.fit(data)

In [24]:
clean_data = cleaner.transform(data)

In [25]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [26]:
clean_data = clean_data.select('label', 'features')

In [29]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [30]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [31]:
spam_detector = nb.fit(training)

In [32]:
test_results = spam_detector.transform(test)

In [33]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-571.51746988582...|[0.99999999999983...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-832.20000433471...|[1.0,1.0177745262...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-659.48112845931...|[1.0,7.7638911198...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-562.57026808665...|[1.0,1.3333907394...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-562.57026808665...|[1.0,1.3333907394...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-666.74225602256...|[1.0,9.4666105225...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-974.94521538547...|[1.0,2.8658548829...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1347.2799309354...|[1.0,1.4503249201...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-614.52505462795...|[0.99999993198625...|       0.0|
|  0.0|(13424,[0

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
acc_eval = MulticlassClassificationEvaluator()

In [36]:
acc = acc_eval.evaluate(test_results)

In [37]:
acc

0.9185170807453417

This is a pretty popular dataset but this notebook provides an implementation of a spam filter in Spark with 91% accuracy.